# Carregando Pacotes

In [1]:
from transformers import pipeline
import torch
import pdfplumber
from transformers import AutoTokenizer

from sentence_transformers import SentenceTransformer

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:


# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

🔧 Rodando em: GPU


# Funções:

In [3]:
def carregar_documento(caminho_do_pdf):

    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages 
            if page.extract_text()  # Ignora páginas sem texto
        )
        texto = " ".join(texto.split())  # Normaliza espaços
    return texto

In [4]:
def dividir_em_chunks_tokenizados(texto):

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

In [5]:
def sliding_window_tokenizados(texto):
    tokens = tokenizer.tokenize(texto)
    janelas = []
    
    for i in range(0, len(tokens), stride):
        janela = tokens[i:i + window_size]
        texto_janela = tokenizer.convert_tokens_to_string(janela)
        janelas.append(texto_janela)
        
        # Para se chegarmos ao final do texto
        if i + window_size >= len(tokens):
            break
    
    return janelas

In [6]:
def recuperar_chunks(pergunta, modelo_vetorizador, divisao_do_texto, k=3):
    # Vetoriza a pergunta
    vetor_pergunta = modelo_vetorizador.encode([pergunta], convert_to_tensor=False)
    
    # Vetoriza todos os chunks
    vetores_chunks = modelo_vetorizador.encode(divisao_do_texto, convert_to_tensor=False)
    
    # Calcula similaridade
    sim = cosine_similarity(vetor_pergunta, vetores_chunks)
    
    # Pega índices dos top-k chunks mais similares
    top_k_indices = np.argsort(sim[0])[::-1][:k]
    
    chunks_relevantes = [divisao_do_texto[i] for i in top_k_indices]
    
    return chunks_relevantes


In [7]:
def gerar_resposta(pergunta, chunks_relevantes):
    contexto = " ".join(chunks_relevantes)
    prompt = f"Contexto: {contexto} \n\nPergunta: {pergunta} \n\nResposta:"
    resposta = modelo_gerador(prompt, max_length=200)
    return resposta[0]['generated_text']


# Variáveis Globais

In [8]:
#Tamanho da Janela de Contexto
window_size=400

#Overlap da Janela de Contexto
stride=100

#Tamanho da Chunk
chunk_size = 400
    
#Overlap da Chunk
overlap = 100

#Tamanho Máximo da Pergunta
tamanho_pergunta = 60

In [9]:
modelo_tokenizador = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"

modelo_vetorizador = SentenceTransformer('all-MiniLM-L6-v2')

modelo_gerador = pipeline("text2text-generation", model="facebook/bart-large", device=device)

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Device set to use cuda:0


# Tratamento do Documento

In [10]:
# Carregar modelo BERTimbau pré-treinado para Q&A em Portugues
tokenizer = AutoTokenizer.from_pretrained(modelo_tokenizador)

qa_pipeline = pipeline(
    "question-answering",
    model= modelo_tokenizador,  # Modelo em Portugues
    tokenizer=modelo_tokenizador,
    device=device
)

Device set to use cuda:0


## Selecionar Divisão do Texto

In [11]:
documento = carregar_documento('Cabelo Cacheado Maior.pdf')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [12]:
#Divisao em Janela Deslizante

divisao_do_texto = sliding_window_tokenizados(documento)

#Divisao em Chunks

#divisao_do_texto = dividir_em_chunks_tokenizados(documento)

In [13]:
# Vetorizar todos os chunks
vetores = modelo_vetorizador.encode(divisao_do_texto, convert_to_tensor=True)

# Processamento

In [14]:
pergunta = "Quais são os tipos de cachos?"

chunks_relevantes = recuperar_chunks(pergunta, modelo_vetorizador, divisao_do_texto)

resposta = gerar_resposta(pergunta, chunks_relevantes)

print("📝 Resposta:", resposta)

Token indices sequence length is longer than the specified maximum sequence length for this model (1861 > 1024). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📝 Resposta: as ação de oportunidade, ou o açõimento, de uma formação ou uma condição, como açúcaro ou como oportuno, com o acesso ao século.Ação como como acontece como funciona. Ação é como ficar como sempre.Alexa como mais comercial. Alexas como provocada. Aquelea comercação. Aseguração aqueleas comerça. Aquestão comercia. Aquelação foi comerciada.Aquelevação física. Elesalimento. Eclesiária. Ecosistema. Económico. Eficácia. Ecológica. Especialidade. Especialidade como empresa. Eso como efeitosa. Efeito. Esteja. Eseja.Esteja ou. Eu.Eu.Ou. O
